In [ ]:
import os
import sys
import tqdm
import pandas as pd

In [ ]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path

from rebadd.evaluate import evaluate_sr_nov_div

In [ ]:
## Test data (navitoclax, ABT-737)
filepath_ref = os.path.join(os.pardir, 'data', 'zinc15', 'zinc15_test.txt')
df_ref = pd.read_csv(filepath_ref, header=None)

referece_smiles_iter = df_ref.iloc[:,0].values.tolist()

print(len(referece_smiles_iter))

In [ ]:
input_dir = 'outputs_7_calculate_properties_generated'

modelnames = [
    'frag+reinforce+scst+offpolicy',
]

filenames = [f'smi_after.csv.{num}' for num in range(10)]

In [ ]:
frames = []

for modelname in modelnames:
    
    for filename in filenames:
        
        filepath = os.path.join(input_dir, modelname, filename)
        
        df = pd.read_csv(filepath)
        
        df = df.loc[:,('smiles', 'bcl2', 'bclxl', 'bclw')]
        df.loc[:,'model'] = modelname
        df.loc[:,'checkpoint'] = filename.split('.')[-1]
        
        frames.append(df)

In [ ]:
data = []

for df in tqdm.tqdm(frames):

    s_sr, s_nov, s_div = evaluate_sr_nov_div(df, referece_smiles_iter, 'bcl2_bclxl_bclw')
    
    data.append({'SR':s_sr, 'Nov':s_nov, 'Div':s_div, 'Model':df.loc[0,'model'], 'Ckpt':df.loc[0,'checkpoint']})

In [ ]:
df_records = pd.DataFrame(data)
df_records.loc[:,'HMean'] = (df_records.loc[:,'SR'] * df_records.loc[:,'Nov'] * df_records.loc[:,'Div']) ** 0.333

df_records

In [ ]:
df_records.groupby('Model').mean(numeric_only=True)

In [ ]:
df_records.groupby('Model').std(numeric_only=True)